In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
%matplotlib inline

In [2]:
df = pd.read_csv('insurance_data.csv')
x_train, x_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)
x_train_scaled = x_train.copy() #copy() makes rerun this block wont calculate new ans
x_train_scaled['age'] = x_train_scaled['age']/100
x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [13]:
class myNN():
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def fit(self, x ,y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(x['age'], x['affordibility'], y, epochs, loss_threshold)
    
    def predict(self, x_test):
        weighted_sum = self.w1*x_test['age'] + self.w2*x_test['affordibility'] + self.bias
        return self.sigmoid_numpy(weighted_sum)
    
    def sigmoid_numpy(self, x): #take array as well
        return 1/(1+np.exp(-x))
    
    def log_loss(self, y_true, y_predicted):
        epsilon = 1e-15
        y_predicted_new = [max(i,epsilon) for i in y_predicted]
        y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
        y_predicted_new = np.array(y_predicted_new)
        return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
        #w1, w2, bias
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_pred = self.sigmoid_numpy(weighted_sum)
            #shape(22,)

            loss = self.log_loss(y_true,y_pred)

            w1d = (1/n)*np.dot(age,(y_pred-y_true))
                               #shape(22,)
            w2d = (1/n)*np.dot(affordibility,(y_pred-y_true))
                               #shape(22,)       
            bias_d = np.mean(y_pred-y_true)

            w1 = w1 - learning_rate*w1d
            w2 = w2 - learning_rate*w2d
            bias = bias - learning_rate*bias_d
            
            if (i+1)%100==0:
                print(f'Epoch:{i+1}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss <= loss_threshold:
                print(f'Epoch:{i+1}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

                break

        return w1, w2, bias

In [14]:
customModel = myNN()
customModel.fit(x_train_scaled, y_train, epochs=8000, loss_threshold=0.4631)

Epoch:100, w1:2.1871944491052364, w2:1.2918774898345378, bias:-1.6533819823304428, loss:0.5395530569489538
Epoch:200, w1:3.4316889691263146, w2:1.4037274192103975, bias:-2.252091504673362, loss:0.5008101251292182
Epoch:300, w1:4.462839550361659, w2:1.4385080284751603, bias:-2.703328362713941, loss:0.4752896446882009
Epoch:367, w1:5.051047623653049, w2:1.4569794548473887, bias:-2.9596534546250037, loss:0.46293944095888917


In [12]:
customModel.predict(x_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64